# Time Series For Relational People

## Install InfluxDB 1.8

The installation is super easy: just download the zip file that you will find in the downloads page at [influxdata.com](https://portal.influxdata.com/downloads/) and expand it to a folder of your choice.

In [ ]:
c:\demo\setup.ps1

In [ ]:
# Turn off progress for download
$ProgressPreference = 'SilentlyContinue'
# Allow all TLS versions
[Net.ServicePointManager]::SecurityProtocol = "Tls12, Tls11, Tls, Ssl3"

# Download InfluxDB 1.8
$tmp = Join-Path -Path C:\temp\ -ChildPath "influxdb.zip"
Invoke-WebRequest -Uri https://dl.influxdata.com/influxdb/releases/influxdb-1.8.10_windows_amd64.zip -OutFile $tmp

# Expand the archive
Expand-Archive -Path $tmp -DestinationPath c:\tick -Force
$influxDBDir = (get-childitem -Path c:\tick -Directory | Where { $_.Name -like 'influx*' }).FullName

# Open folder
Start-Process $influxDBDir

## Run InfluxDB

Starting InfluxDB is as simple as running the daemon executable `influxd.exe.` Of course we could change a lot of options in the config file, but we are running the defaults for this session.

In [ ]:
Start-Process c:\tick\influxdb-1.8.10-1\influxd.exe

## Install InfluxDB as a service

InfluxDB does not have the ability to be run as a service directly, but it needs a wrapper service like [NSSM](http://nssm.cc/) (the **N**on-**S**ucking **S**ervice **M**anager)

In [ ]:
# Turn off progress for download
$ProgressPreference = 'SilentlyContinue'
# Allow all TLS versions
[Net.ServicePointManager]::SecurityProtocol = "Tls12, Tls11, Tls, Ssl3"

# Download NSSM
$tmp = Join-Path -Path C:\temp\ -ChildPath "nssm.zip"
Invoke-WebRequest -Uri http://nssm.cc/release/nssm-2.24.zip -OutFile $tmp

# Expand the archive and show extracted files
Expand-Archive -Path $tmp -DestinationPath c:\tick -Force
Start-Process C:\tick\nssm-2.24

In [ ]:
if(get-service influxdb) {
    get-service influxdb | stop-service
    sc.exe delete influxdb
}
# create a service for influxdb
start-process C:\tick\nssm-2.24\win64\nssm.exe -argumentlist "install","influxdb"

In [ ]:
# start the service
Get-Service influxdb | Where-Object {$_.Status -ne "running"} | Start-Service
Get-Service influxdb

## Working With InfluxDB

You can query InfluxDB using the CLI or the http API

In [ ]:
code C:\Demo\TimeSeries.psm1

In [ ]:
Import-Module C:\Demo\TimeSeries.psm1 -Force
Invoke-InfluxQLCmd -Query "SHOW DATABASES"

In [ ]:
Invoke-InfluxQLCmd -Query "CREATE DATABASE demo" -method "post" | Out-Null
& c:\tick\influxdb-1.8.10-1\influx.exe -execute 'SHOW DATABASES'

## Writing data to InfluxDB

You can use the write endpoint or you can use the CLI and the INSERT command

In [ ]:
# Write data using the /write endpoint
Write-LineProtocol -Body "CPUHistory,server_name=ACCOUNTING SQLServerCPU=31,OtherCPU=7 1652386888000000000" -Database "demo"

C:\tick\influxdb-1.8.10-1\influx.exe -database demo -execute 'INSERT CPUHistory,server_name=CRM SQLServerCPU=43,OtherCPU=8 1652386888000000000'

# Query list of measurements
# A new measurement has been created automatically
Invoke-InfluxQLCmd -Query "SHOW MEASUREMENTS" -Database "demo"  | Out-Default

In [ ]:
# Write data in batches
Write-LineProtocol -Database "demo" -Body "
CPUHistory,server_name=ACCOUNTING SQLServerCPU=31,OtherCPU=7 1652391123000000000
CPUHistory,server_name=ACCOUNTING SQLServerCPU=48,OtherCPU=9 1652391137000000000
CPUHistory,server_name=ACCOUNTING SQLServerCPU=53,OtherCPU=7 1652391145000000000
CPUHistory,server_name=ACCOUNTING SQLServerCPU=47,OtherCPU=3 1652391150000000000
CPUHistory,server_name=CRM SQLServerCPU=21,OtherCPU=3 1652391123000000000
CPUHistory,server_name=CRM SQLServerCPU=38,OtherCPU=2 1652391137000000000
CPUHistory,server_name=CRM SQLServerCPU=43,OtherCPU=4 1652391145000000000
CPUHistory,server_name=CRM SQLServerCPU=37,OtherCPU=5 1652391150000000000
"

## Writing data with Telegraf

Telegraf is the data collection agent for InfluxDB. Let's play with it a bit.

In [ ]:
# Turn off progress for download
$ProgressPreference = 'SilentlyContinue'
# Allow all TLS versions
[Net.ServicePointManager]::SecurityProtocol = "Tls12, Tls11, Tls, Ssl3"

$tmp = Join-Path -Path C:\temp\ -ChildPath "telegraf.zip"
Invoke-WebRequest -Uri https://dl.influxdata.com/telegraf/releases/telegraf-1.22.3_windows_amd64.zip -OutFile $tmp
Expand-Archive -Path $tmp -DestinationPath c:\tick -Force

$telegrafDir = (get-childitem -Path c:\tick -Directory | Where { $_.Name -like 'telegraf*' }).FullName

In [ ]:
#generate a filtered configuration (like the default one) and write it to a file
& $telegrafDir\telegraf.exe --input-filter sqlserver --output-filter influxdb config | 
    ForEach-Object { $_.Replace('Server=192.168.1.10;Port=1433;User Id=<user>;Password=<pw>;app name=telegraf;log=1;', 'Server=127.0.0.1;Port=1433;app name=telegraf;log=1;') } |
    ForEach-Object { $_.Replace('# urls = ["http://127.0.0.1:8086"]', 'urls = ["http://127.0.0.1:8086"]') } |
    Set-Content -Encoding UTF8 -Path $telegrafDir\telegraf_sqlserver_counters.conf    

& code $telegrafDir\telegraf_sqlserver_counters.conf

In [ ]:
# test the config file
& $telegrafDir\telegraf.exe -config $telegrafDir\telegraf_sqlserver_counters.conf -test

In [ ]:
# let's start telegraf
start-process $telegrafDir\telegraf.exe -ArgumentList @("--config","$telegrafDir\telegraf_sqlserver_counters.conf")

In [ ]:
# let's check if it is writing data
Invoke-InfluxQLCmd -Database "telegraf" -Query "SHOW MEASUREMENTS"

## Querying data

InfluxQL is the v1 language available in InfluxDB. Less powerful but definitely more readable for relational people

In [ ]:
# Example 1: simple SELECT query
# It looks like a table with rows and columns, but that's not the case
# It's actually multiple series combined in a tabular fashion

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT *
    FROM CPUHistory
"

In [ ]:
# Example 2: simple SELECT query with WHERE clause
# very similar to SQL Server

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT *
    FROM CPUHistory
    WHERE server_name = 'ACCOUNTING'
"

In [ ]:
# Example 3:  it's case sensistive!

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT *
    FROM CPUHistory
    WHERE server_name = 'accounting'
" | Out-Default



# Example 4:  it works with fields and tags not in the measurement

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT ghost_field
    FROM CPUHistory
    WHERE field_which_is_not_there = 'something'
" | Out-Default

In [ ]:
# Example 5:  WHERE clause on time

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT *
    FROM CPUHistory
    WHERE time > now()-1d
" 

In [ ]:
# Example 5:  GROUP BY clause and column aliases

Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT MEAN(SQLServerCPU) AS avg_cpu
    FROM CPUHistory
    WHERE time > now() - 1d
    GROUP BY server_name
"

In [ ]:
# Example 6:  GROUP BY time

Invoke-InfluxQLCmd -Database "telegraf" -Query @"
    SELECT mean("sqlserver_process_cpu") AS "SQL Server CPU", 
           mean("other_process_cpu") AS "Other CPU" 
    FROM sqlserver_cpu
    WHERE time > now() - 1d
    GROUP BY time(10m)
    FILL(none)
"@

In [ ]:
# Example 7:  LIMIT and SLIMIT

# limits points INSIDE A SERIES
Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT MEAN(SQLServerCPU) AS avg_cpu
    FROM CPUHistory
    WHERE time > now() - 1d
    GROUP BY server_name
    LIMIT 1
" | out-default

# limits the number of series
Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT MEAN(SQLServerCPU) AS avg_cpu
    FROM CPUHistory
    WHERE time > now() - 1d
    GROUP BY server_name
    SLIMIT 1
" | out-default

## Modifying data

UPDATE works by uploading the data again. Not the brightest idea performance-wise, but it's doable. 

DELETE works like in SQL

In [ ]:
# Let's set "OtherCPU" to 0 for all points
Write-LineProtocol -Database "demo" -Body "
CPUHistory,server_name=ACCOUNTING OtherCPU=0 1652391123000000000
CPUHistory,server_name=ACCOUNTING OtherCPU=0 1652391137000000000
CPUHistory,server_name=ACCOUNTING OtherCPU=0 1652391145000000000
CPUHistory,server_name=ACCOUNTING OtherCPU=0 1652391150000000000
CPUHistory,server_name=CRM OtherCPU=0 1652391123000000000
CPUHistory,server_name=CRM OtherCPU=0 1652391137000000000
CPUHistory,server_name=CRM OtherCPU=0 1652391145000000000
CPUHistory,server_name=CRM OtherCPU=0 1652391150000000000
"

In [ ]:
# Let's verify that the data has been updated
Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT server_name, OtherCPU
    FROM CPUHistory
" 

In [ ]:
Invoke-InfluxQLCmd -Database "demo" -Query "
    DELETE 
    FROM CPUHistory
" 

In [ ]:
# Let's verify that the data has been deleted
Invoke-InfluxQLCmd -Database "demo" -Query "
    SELECT *
    FROM CPUHistory
" 

## Creating dashboards with Grafana

- Download Grafana
- Extract the archive
- Run Grafana

In [ ]:
# Download and install Grafana
$ProgressPreference = 'SilentlyContinue'
[Net.ServicePointManager]::SecurityProtocol = "Tls12, Tls11, Tls, Ssl3"

$tmp = Join-Path -Path C:\temp\ -ChildPath "grafana.zip"
Invoke-WebRequest -Uri https://dl.grafana.com/enterprise/release/grafana-enterprise-8.5.2.windows-amd64.zip -OutFile $tmp
Expand-Archive -Path $tmp -DestinationPath c:\tick -Force
$grafanaDir = (get-childitem -Path c:\tick -Directory | Where { $_.Name -like 'grafana*' }).FullName

# start grafana
start-process $grafanaDir\bin\grafana-server.exe -ArgumentList @("-homepath","$grafanaDir")

In [ ]:

# open browser and connect to Grafana
start-process http://localhost:3000

### **Create a datasource:**

Go to data sources, add an InfluxDB data source that points to `http://localhost:8086` and database `telegraf`

### **Create a dashboard:**

Create a new dashboard, add a panel and enter this query in the panel query editor:

```
SELECT mean("sqlserver_process_cpu") AS "SQL Server", 
       mean("other_process_cpu") AS "Other" 
FROM "sqlserver_cpu" 
WHERE $timeFilter 
GROUP BY time($__interval) 
```